# Memahami Data

“Dari Wikipedia bahasa Indonesia, ensiklopedia bebas Bunga iris (Neomarica longifolia; sinonim: Trimezia longifolia)[1] adalah terna (herb) yang berkembang biak dengan menggunakan rhizome (rimpang) dan termasuk ke dalam keluarga Iridaceae.” (Kontributor dari proyek Wikimedia., 2021)

Bunga iris dapat diperjualbelikan secara bebas dan dapat digunakan sebagai tanaman hias. Warna dan bentuknya menarik dan dapat ditanam dalam pot maupun di tanam langsung di tanah halaman rumah. Selain menjadi tanaman hias, tanaman ini dapat dimanfaatkan juga sebagai tanaman obat. Tanaman ini bisa digunakan untuk mengobati hepatitis, mual, sembelit, rematik, dan gusi bengkak.

## Pada pekerjaan ini kami akan mendeskripsikan data iris

- Deskripsi fitur

- Amati missing value

- Inputasi missing value dengan metode KNN dan Mean lalu dibandingkan

- Hitung akurasi dan perbandingan tiap metode

<img src="image-20240308-153538.png" width="" align="" />

### Inisialisasi

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

## Menampilkan Data

### Ambil data sepal iris dari MySQL

In [ ]:
df_sepal = _deepnote_execute_sql('SELECT * FROM iris', 'SQL_86FFEF30_2CE7_45FB_B840_F36435E1CC93', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_sepal

,Id,SepalLengthCm,SepalWidthCm,Species
0,1,5.1,3.5,Iris-setosa
1,2,4.9,3.0,Iris-setosa
2,3,4.7,3.2,Iris-setosa
3,4,4.6,3.1,Iris-setosa
4,5,5.0,3.6,Iris-setosa
...,...,...,...,...
145,146,6.7,3.0,Iris-virginica
146,147,6.3,2.5,Iris-virginica
147,148,6.5,3.0,Iris-virginica
148,149,6.2,3.4,Iris-virginica


### Ambil data petal iris dari PostgreSQL

In [ ]:
df_petal = _deepnote_execute_sql('SELECT * FROM iris', 'SQL_843DF3DB_71CE_4E04_AB63_48BA92D27513', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_petal

,Id,PetalLengthCm,PetalWidthCm,Species
0,1,1.4,0.2,Iris-setosa
1,2,1.4,0.2,Iris-setosa
2,3,1.3,0.2,Iris-setosa
3,4,1.5,0.2,Iris-setosa
4,5,1.4,0.2,Iris-setosa
...,...,...,...,...
145,146,5.2,2.3,Iris-virginica
146,147,5.0,1.9,Iris-virginica
147,148,5.2,2.0,Iris-virginica
148,149,5.4,2.3,Iris-virginica


### Penggabungan data dari dua sumber yang berbeda

In [ ]:
# concat two dataframe
df = pd.concat([df_sepal.drop(['Species'], axis=1), df_petal.drop(['Id'], axis=1)], axis=1, join='inner')

df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


## Deskripsi Data

### Penjelasan fitur

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


Berdasarkan output dari kode program diatas. kita dapat mengetahui keterangan mengenai Tipe Data dari masing masing fitur. terdapat empat fitur dan satu class. dijelaskan sebagai berikut : 

- SepalLengthCm merupakan fitur dengan tipe data numerik yang merupakan hasil dari pengukuran panjang kelopak sepal dari bunga iris.

- SepalWidhtCM merupakan fitur dengan tipe data numerik yang merupakan hasil dari pengukuran lebar kelopak sepal dari dari bunga iris.

- PetalLengthCm merupakan fitur dengan tipe data numerik yang merupakan hasil dari pengukuran panjang kelopak petal dari bunga iris.

- PetalWidthCm merupakan fitur dengan tipe data numerik yang merupakan hasil pengukuran dari lebar kelopak sepal dari bunga iris.

- Species merupakan class dengan tipe data kategori yang merupakan klasifikasi jenis bungan iris.

### Penjelasan class

In [ ]:
df.value_counts('Species')

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

Dari kode program diatas. dapat diketahui bahwa terdapat tiga jenis bunga iris berdasarkan data yang sudah kita analisa.

## Percobaan Missing Value

Pada tahapan ini akan melakukan sebuah bercobaan dengan menghapus lima value dari beberapa baris. sehingga secara tidak langsung kita melakukan sebuah simulasi menghadapi sebuah data yang memiliki missing value.

### Menghapus value dari baris

In [ ]:
df.loc[df["Id"] == 50, 'SepalLengthCm'] = None
df.loc[df["Id"] == 100, 'SepalLengthCm'] = None
df.loc[df["Id"] == 150, 'SepalLengthCm'] = None

# dua fitur pada 
df.loc[df["Id"] == 25, ['SepalLengthCm','PetalWidthCm']] = None
df.loc[df["Id"] == 75, ['SepalLengthCm','PetalWidthCm']] = None

df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


### Menampilkan fitur yang memiliki missing value

In [ ]:
df.isnull().sum()

Id               0
SepalLengthCm    5
SepalWidthCm     0
PetalLengthCm    0
PetalWidthCm     2
Species          0
dtype: int64

Dari kode diatas dapat dilihat bahwa terdapat missng value pada fitur SepalLengthCm dan PetalWidthCm. pada fitur SepalLengthCm terdapat lima baris yang memiliki missing value. dan pada fitur PetalWidthCm terdapat dua baris yang memiliki missing value

# Pra-pemrosesan Data (Data Preprocessing)

Data preprocessing dalam data mining sangat penting. Pada tahapan ini, akan dilakukan pengolahan data mentah (Raw data) yang nantinya akan diubah menjadi data yang  bisa diolah untuk di analisa lebih lanjut.

Ada 4 tahapan untuk melakukan pra-pemrosesan data, yaitu :

- Pembersihan Data (Data Cleaning)

Pada tahapan ini. Data mentah (Raw Data) yang diperoleh harus dilakukan seleksi dengan menghapus data yang tidak relevan, lengkap, dan tidak akurat. Hal ini berguna untuk mengurangi kesalahpahaman data.

- Penggabungan Data (Data Integration)

Data dari sumber yang berbeda digabungkan menjadi satu dataset. karena itu, kita harus mengecek sumber-sumber data agar supaya memiliki format yang sama.

- Tranformasi Data (Data Tranformation)

Setelah proses penggabungan data dari berbagai sumber data. Selanjutnya akan dilakukan proses tranformasi data. Hal ini dilakukan karena kemungkinan data yang diambil memiliki perbedaan format. karena itu agar memudahkan ketika diolah, Data harus disamakan formatnya.

- Pengurangan Data (Data Reduction)

Terakhir yang dilakukan adalah mengurangi data. Maksudnya adalah mengurangi sampel data yang diambil. walupun dilakukan pengurangan sampel data, pengurangan ini tidak boleh mengubah hasil analisa data. Ada tiga cara yang dilakukan. Pertama, Mengurangi dimensi. Kedua, pengurangan jumlah data. Dan ketiga, kompresi data.

## Pembersihan Data (Data Cleaning)

### Seleksi fitur

Seleksi fitur dilakukan untuk menghapus kolom / fitur yang tidak dibutuhkan. dalam kasus ini, kolom Id dihapus karena tidak dibutuhkan dalam komputasi.

In [ ]:
df = df.drop(['Id'], axis=1)

df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


### Merubah nama kolom

Dataset biasanya memiliki nama kolom / fitur yang kurang manusiawi sehingga diperlukan peruahan nama agar memudahkan proses analisis.

In [ ]:
df = df.rename(columns={'SepalLengthCm' : 'Sepal Length', 'SepalWidthCm' : 'Sepal Width', 'PetalLengthCm' : 'Petal Length', 'PetalWidthCm' : 'Petal Width'})

df.head(5)

,Sepal Length,Sepal Width,Petal Length,Petal Width,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Menghapus baris duplikat

Dataset yang baik adalah dataset yang memiliki jumlah data yang banyak dan beragam. Banyaknya data yang sama tidak akan membuat model memilki akurasi yang tinggi.

In [ ]:
# cek jumlah baris dan kolom
df.shape

(150, 5)

In [ ]:
# cek jumlah baris dan kolom yang memiliki data duplikat
duplicate_rows_df = df[df.duplicated()]

print('number of duplicate rows : ', duplicate_rows_df.shape)

number of duplicate rows :  (3, 5)


Dari kode program diatas, dapat diketahui bahwa terdapat tiga baris yang memilki data yang sama

In [ ]:
# menghapus duplikasi data
df = df.drop_duplicates()

print('number of rows after drop duplicated rows', df.shape)

number of rows after drop duplicated rows (147, 5)


Berdasarkan output kode proram diatas, jumlah baris yang awalnya 150 baris menjadi 147 baris. hal ini membuktikan bahwa kita sudah berhasil menghapus baris yang memiliki data yang sama.

### Penanganan missing value

Data yang hilang atau kosong harus ditangani dengan serius. Hal ini akan merusak dan menganggu pemodelan dataset.Data yang hilang atau kosong berasal dari sumber data yang tidak bekerja secara benar. bisa saja dari mesin yang awalnya bekerja secara baik namun diproses pengerjaan mengalamai error. Ataupun bila data yang didapatkan dari manusisa, bisa saja data yang hilang disebabkan oleh human error atau privatisasi data. 

Dalam menangani missing value ada beberapa cara. umumnya bisa dengan dihapus ataupun di isi dengan metode metode tertentu. dalam kasus ini, kita akan mencoba melakukan hapus data dan mengisi data dengan metode menghitung rata-rata (mean) dan metode K Nearest Neightbour (KNN).

Penanganan missing value dengan menghapus baris

In [ ]:
# menampilkan baris dengan missing value
df.loc[df['Sepal Length'].isnull() | df['Sepal Width'].isnull() | df['Petal Length'].isnull() | df['Petal Width'].isnull()]

,Sepal Length,Sepal Width,Petal Length,Petal Width,Species
24,NaN,3.4,1.9,NaN,Iris-setosa
49,NaN,3.3,1.4,0.2,Iris-setosa
74,NaN,2.9,4.3,NaN,Iris-versicolor
99,NaN,2.8,4.1,1.3,Iris-versicolor
149,NaN,3.0,5.1,1.8,Iris-virginica


Dalam kasus ini terdapat lima baris data yang memiliki missing value. pada baris 24 dan 74. terdapat dua kolom yang memiliki missing value. baris ini kita hapus karena kita kehilangan 2 data fitur dari 4 fitur. atau dapat dibilang kita kehilangan 50% informasi fitur pada fitur.

In [ ]:
# menghapus baris baris dengan missing value pada kolom 'Petal Width'
df.dropna(subset=['Petal Width'], inplace=True)

In [ ]:
df

,Sepal Length,Sepal Width,Petal Length,Petal Width,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


Pada kode diatas, kita sudah berhasil menghapus baris 24 dan 74. selanjutnya kita akan melakukan imputasi data pada baris 99 dan 149. kali ini kita kan melkukannya dengan menggunakan dua metode, yaitu metode mean atau rata-rata dan metode KNN. lalu pada proses berikutnya akan dilakukan perbandingan akurasi dari kedua metode tersebut.

Penanganan missing value dengan metode nilai rata-rata (Mean)

Pada langkah sebelumnya kit sudah berhasil menghapus baris, pada langkah kali ini akan dilakukan imputasi missing value dengan metode nilai rata-rata (Mean). yaitu dengan menjumlahkan seluruh baris nilai fitur dan dilakukan operasi pembagian dengan jumlah baris. maka akan ditemukan hasil rata-rata nilai fitur. adapun rumusnya sebagai berikut.

$ \bar{x} = \sum_{i = 1}^{n} \frac{x_{i}}{n} $

In [ ]:
# membuat copy dataframe pada variabel lain
df_mean = df.copy(deep=True)

# mencari nilai mean
mean = round(df_mean['Sepal Length'].mean(), 1)

print('nilai mean adalah = ', mean)

# imputasi missing value dengan mean
df_mean['Sepal Length'].fillna(mean, inplace=True)

df_mean

nilai mean adalah =  5.9


,Sepal Length,Sepal Width,Petal Length,Petal Width,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


Berdasarkan kode program diatas. kita membuat duplikasi dataframe yang nantinya untuk menyipan dataframe yang menggunakan metode mean. kemudian mencari nilai mean dan timpa atau gantikan baris yang memiliki missing value dengan nilai mean.

Penanganan missing value dengan metode K Nearest Neightbour (KNN)

Nearest Neightbour jika di translasikan menjad bahasa indonesia adalah tetangga terdekat. Dan  K yang dimaksud adalah jumlah tetangga tedekat. Jadi dapat di definisikan bahwa K Nearest Neightbour (KNN) merupakan metode dengan mencari jumlah tetangga terdekat dan akan dilakukan perbandingan class untuk memprediksi klasifikasi data baru. Adapun cara kerjanya bisa disimak dibawah ini.

<img src="image-20240311-005422.png" width="" align="" />

Cara kerja K Nearest Neightbour (KNN)

0. Look at the data (Melihat pada data)

Berdasarkan pada gambar diata. node atau tiitk berwarna abu-abu merupakan data baru. data ini akan nantinya akan diklasifikasikan dengan class-class pada digram. Jika kita perhatikan, terdapat tiga node yang berbeda warna, yaitu kuning, hijau, dan jingga. Hal ini berarti terdapat tiga class dalam diagram. node yang berwarna abu-abu akan diprediksi dengan algoritma KNN untuk menentukan node ini termasuk ke dalam salah satu class.

.1. Calculate distances (Kalkulasi jarak)

Node baru akan dilakukan kalkulasi jarak terhadap node-node yang lain. dalam mengukur jarak bisa dilakukan dengan beberapa metode beriku :

Euclidean Distance

<img src="image-20240311-200655.png" width="50%" align="" />

Merupakan teknik yang paling populer untuk melakukan perhitungan jarak fitur dengan tipe data numerik. Untuk rumus pengukuran jarak adalah sebagai berikut ini.

$ d(P,Q) = \sqrt{\sum_{i =1}^{n} (p_{i}-q_{i})^{2}}$
$ d(P,Q) = \sqrt{(p_{1}-q_{1})^{2}} + (p_{2}-q_{2})^{2} + ..... + (p_{n}-q_{n})^{2}$

Manhattan Distance

<img src="image-20240311-201408.png" width="" align="" />

Manhattan distance juga begitu populer selain euclidean distance. Jika kita perhatikan gambar diatas yang merupakan perbandingan antara euclidean distance dan manhattan distance. Dapat dilihat jika euclidean distance merupakan pengukuran jarak terobos antara dua titik kordinat. Sedangkan manhattan berbeda. teknik manhattan berjalan pada grid dan tidak menerobos seperti euclidean. Rumus perhitungan jarak dapat diihat berikut ini.

$ d(P, Q) = (p_{i} - q{i})^2 $

2. Find neightbour (Mencari tetangga)

Setelah melakukan perhitungan jarak terhadap fitur ataupun node. Kemudian akan dilakukan perankingan jarak. mulai dari yang terdekat hingga yang terjauh.

3. Vote on label (Memilih label)

Selanjutnya setelah ditemukan hasil dan perankingan jarak. Akan  dilakukan penentuan fitur terdekat atau nilai K. dalam KNN nilai K boleh ditentukan berapapun atau bebas. semisal K adalah 5, maka kita akan mengambil 5 fitur atau node yang terdekat. Selanjutnya akan dilakukan penentuan class pada node baru. Misal K adalah 5. maka akan diambil 5 fitur, kemudian sebagai penetuan class untuk node baru. bisa ditentukan dengan melihat nilai class yang paling banyak muncul pada kelima data yang diambil. semisal dalam kelima data ini class yang paling banyak muncul adalah class anggrej bulan. maka nilai class dari data baru adalah anggrek bulan.

Imputasi missing value dengan KNNImputer

In [ ]:
from sklearn.impute import KNNImputer

# duplikasi dataframe
df_knn = df.copy(deep=True)

# simpan class species
df_species = df_knn[['Species']]

list_species = df_species.values.tolist()

#hapus kolom yang tidak diperlukan
df_knn = df_knn.drop(['Species'], axis=1)

# old_value = df_knn.drop(['Species'], axis=1).to_numpy()

# inisialisasi KNNImputer
imputer = KNNImputer(n_neighbors=3)

# simpan kalkulasi
result = imputer.fit_transform(df_knn)

result = result.tolist()

# gabung hasil dengan class
for index in range(len(result)):
    result[index].append(list_species[index][0])

# buat dataframe
df_knn = pd.DataFrame(result, columns=['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width', 'Species'])

df_knn

,Sepal Length,Sepal Width,Petal Length,Petal Width,Species
0,5.100000,3.5,1.4,0.2,Iris-setosa
1,4.900000,3.0,1.4,0.2,Iris-setosa
2,4.700000,3.2,1.3,0.2,Iris-setosa
3,4.600000,3.1,1.5,0.2,Iris-setosa
4,5.000000,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
140,6.700000,3.0,5.2,2.3,Iris-virginica
141,6.300000,2.5,5.0,1.9,Iris-virginica
142,6.500000,3.0,5.2,2.0,Iris-virginica
143,6.200000,3.4,5.4,2.3,Iris-virginica


## Deteksi Outlier (Local Outlier Factor)

Konsep Local Outlier Factor

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1dd64262-f415-4de7-aded-3acfb4c0cee9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>